# What is a linker script?

- It is a text file that explains how different sections of the object files should be merged to create an output file
- Linker and locator 
- Linker script also includes code and data memory address and size information
- These scripts are written using _GNU linker command language_
- GNU linker script has the file extension of `.ld`
- You must supply linker script at the linking phase to the linker using `-T` option

# Basic linker script commands
 * Entry
 * Memory
 * Sections
 * Keep
 * Align
 * AT>

 ## Entry
 * This command is used to set the Entry point address information in th header of the final elf generated file
 * For a microcontroller, the `Reset_Handler` is the entry point into the application. This is the first piece of code that executes after a processor reset
 * The debugger uses this information to locate the first function to execute
 * This is not a mandatory command to use, but required when you debug the elf file using the debugger (GDB)
 * Syntax: Entry(_symbol_name_); Example: `Entry(Reset_Handler)`

 ## Memory 

 - Allows you to describe the different memories present in the target processor and their start address and size information
 - The linker uses information mentioned in this command to assign addresses to merged sections
 - Helps in data size verifications as this helps to calculate the total code and data memory so far consumed
 - Fine tune various memories available in the target and allow different sections to occupy different memory areas
 - Typiclaly one linker scipt has one memory command
### Syntax:
```
MEMORY
{
    name(attr) : ORIGIN = origin, LENGTH = len
}
```
label defines name of the memory region, will be ereferenced by other parts of the linker script
Origin: defines the start of the memory region
Length: Defines the length information

| Attribute Letter    | Meaning |
| -------- | ------- |
| R  | Read only sections    |
| W | Read and write sections     |
| X    | Sections containing executable code    |
| A    | Allocated sections    |
| I    | Initialized sections    |
| L    | Same as "I"    |
| !    | Invert the sense of any of the following attributes    |



![image](images/linkerMemoryMap1.png)

In the above code snippet we have taken the STM32F446RE microcontroller's memories and wrote them as three sections, namely, 
1. FLASH - with read (r) and executable (x) access only. We dont write to it, so we did not grant access
2. SRAM1 - with (rwx), meaning read, write and executable access
3. SRAM2 - with (rwx), meaning read, write and executable access

So, from the datasheet/technical refernce manual for the microcontroller, you should find
* How many memory spaces exist?
* Where are their starting origins?
* What is their length?

Also, the presenter decided to squish SRAM1 and SRAM2 into one memory space 

![image](images/linkerMemoryMap2.png)

So, we basically added `116 + 16 = 132 - 4 = 128`. So, why did we subtract 4KB? 

### Why did we subtract 4KB?
This is an important learning moment. Some microcontrollers have memory layout conventions that reserve some code memories or have other special functions. If we do not wish to use these memories, we exclude them from linker script. So, for the STM32 architecure, the manual says...

![image](images/embeddedSRAM_Note.png)

- This is why we subtracted 4KB as we are not writing to it, or interacting with it
- It’s not a linker script syntax requirement, but a platform-specific memory layout convention
- So, you need to read the microcontroller data sheet to assess whether you have such special features in the SRAM spaces 
- If you wish to intearct with this "back SRAM", you need to write it separately in the linker script

# Sections command
- `SECTIONS` command is used to create different output sections in the final elf executable generated
- This is an important commmand to instruct the linker script on how to merge the input sections to yield an output section
- This command also controls the order in which different output sections appear in the elf file generated
- You also mention the placement of a section in a memory region. For example, you instruct the linker to place the `.text` section in the `FLASH` memory region, which is described by the `MEMORY` command

![image](images/basic_SECTIONS_Command_Usage.png)

- In the above code, two sections namely `.text` and `.data` are declared
- This command is telling the linker to generate two sections `.text` and `.data` in the final `.elf` file
- You can give any name to labels like `.text`, but it is a good idea to follow standard naming conventions 
- But we know that according to gcc compilation, the memory sections names are standardized as `.data`, `.bss`, `.text`, `.rodata` etc...
- Next the end of each section `>(vma) AT>(lma)` gives the section placement information in the memory. They satdn for Virtual Memory Address and Load Memory Address

> Note that there are many files that have the `.text` section when they pass the compilation phase and manifest as `.o` files. So, the `.text` in linker takes all these individual sections and distills them into one contiiguous memory section. Same logic follows for all other sections.

Below image clarifies the above statement
![image](images/variousObjectsandTheirSections.png)